In [ ]:
from __future__ import division, print_function, absolute_import

import sqlite3
import tflearn as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
conn = sqlite3.connect('surge_data.db')
c = conn.cursor()

In [ ]:
# this gets run when I'm done working for the session
# conn.close()

In [ ]:
for row in c.execute('SELECT * FROM surge'):
    print (row)
    break

In [ ]:
def build_dataframes_for_points(df):
    """
    makes different dataframes that only have data from one surge point
    """
    point_list = [0, 1, 2, 3, 14, 15, 16, 12, 13, 24, 25, 26, 27, 28, 17, 29]
    return [df[df.point == point].reset_index(drop=True)[:31184] for point in point_list]

# df_0,df_1,df_2,df_3,df_14,df_15,df_16,df_12,df_13,df_24,df_25,df_26,\
#                                                         df_27,df_28,df_17,df_29 = build_dataframes_for_points(df)
sin_cos = np.array([np.array(d_f.surge) for d_f in build_dataframes_for_points(df)]).T

In [ ]:
steps_of_history = 20
steps_in_future = 3

y = sin_cos[steps_of_history+steps_in_future-1:,:]
X = sin_cos[:len(y),:]
my_x = np.empty([len(y),steps_of_history,16])

for i, _ in enumerate(X):
    time_chunk = X[i:i+steps_of_history,:]
    if time_chunk.shape == (steps_of_history, 16):
        my_x[i,:,:] = time_chunk
    else:
        my_x[i,:,:] = np.ones((steps_of_history,16))

trainX, testX = my_x[:-1000,:], my_x[-1000:,:]
trainY, testY = y[:-1000,:], y[-1000:,:]

In [ ]:
# Build my neural net
net = tf.input_data(shape=[None, steps_of_history, 16])
net = tf.lstm(net, 256, return_seq=False)
net = tf.dropout(net, 0.5)

# net = tflearn.lstm(net, 128, return_seq=False)
# net = tflearn.dropout(net, 0.5)

net = tf.fully_connected(net, 16, activation='linear')
net = tf.regression(net, optimizer='sgd', loss='mean_square', learning_rate=0.3)

In [ ]:
# Training
model = tf.DNN(net, clip_gradients=0.0, tensorboard_verbose=0)
model.fit(trainX, trainY, n_epoch=30, validation_set=0.1, batch_size=128)

predictY = np.array(model.predict(testX))

In [ ]:
corrected_predictions = np.zeros(predictY.shape)

for i, time in enumerate(predictY):
    for j, point in enumerate(time):
        if point <=1.05:
            corrected_predictions[i,j] = 1.0
        else:
            corrected_predictions[i,j] = point

In [ ]:
np.mean((predictY - testY)**2), np.mean((corrected_predictions - testY)**2)

In [ ]:
predictY = corrected_predictions

In [ ]:
# Plot the results
plt.figure(figsize=(20,25))

plt.subplot(811)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,0], 'k-', label='Actual')
plt.plot(predictY[:,0], 'c-', label='Forecast')
plt.legend()

plt.subplot(812)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,1], 'k-', label='Actual')
plt.plot(predictY[:,1], 'c-', label='Predicted')
plt.legend()

plt.subplot(813)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,2], 'k-', label='Actual')
plt.plot(predictY[:,2], 'c-', label='Forecast')
plt.legend()

plt.subplot(814)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,3], 'k-', label='Actual')
plt.plot(predictY[:,3], 'c-', label='Predicted')
plt.legend()

plt.subplot(815)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,4], 'k-', label='Actual')
plt.plot(predictY[:,4], 'c-', label='Forecast')
plt.legend()

plt.subplot(816)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,5], 'k-', label='Actual')
plt.plot(predictY[:,5], 'c-', label='Predicted')
plt.legend()

plt.subplot(817)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,6], 'k-', label='Actual')
plt.plot(predictY[:,6], 'c-', label='Forecast')
plt.legend()

plt.subplot(818)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,7], 'k-', label='Actual')
plt.plot(predictY[:,7], 'c-', label='Predicted')
plt.legend()

plt.show()

In [ ]:
# Plot the results
plt.figure(figsize=(20,25))

plt.subplot(811)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,8], 'k-', label='Actual')
plt.plot(predictY[:,8], 'c-', label='Forecast')
plt.legend()

plt.subplot(812)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,9], 'k-', label='Actual')
plt.plot(predictY[:,9], 'c-', label='Predicted')
plt.legend()

plt.subplot(813)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,10], 'k-', label='Actual')
plt.plot(predictY[:,10], 'c-', label='Forecast')
plt.legend()

plt.subplot(814)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,11], 'k-', label='Actual')
plt.plot(predictY[:,11], 'c-', label='Predicted')
plt.legend()

plt.subplot(815)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,12], 'k-', label='Actual')
plt.plot(predictY[:,12], 'c-', label='Forecast')
plt.legend()

plt.subplot(816)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,13], 'k-', label='Actual')
plt.plot(predictY[:,13], 'c-', label='Predicted')
plt.legend()

plt.subplot(817)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,14], 'k-', label='Actual')
plt.plot(predictY[:,14], 'c-', label='Forecast')
plt.legend()

plt.subplot(818)
plt.title('History='+str(steps_of_history)+', Future='+str(steps_in_future))
plt.plot(testY[:,15], 'k-', label='Actual')
plt.plot(predictY[:,15], 'c-', label='Predicted')
plt.legend()

plt.show()